In [ ]:
%pip install -r PredictionModel/requirements.txt

In [ ]:
#!python PredictionModel/runPredModel.py patient-data/p001/p001.jpg
from PredictionModel.runPredModel import scanallxrays
import os
import pprint

pdir = "patient-data"
for p in os.listdir(pdir):
    patient_dir = os.path.join(pdir, p)
    print(f"patient {p}: {patient_dir}")
    pprint.pprint(scanallxrays(patient_dir))

In [ ]:
from PredictionModel.convertDcm import convertall

convertall("patient-data/p001")

In [1]:
from PredictionModel.heatmaps import genheatmap
from PredictionModel.runPredModel import diseases


img_path = 'patient-data/p001/p001.dcm'
# pathology = 'Atelectasis'

for ds in diseases:
    print(f"Disease: {ds}")
    genheatmap(img_path, ds)
    break

Disease: Atelectasis
Setting XRayResizer engine to cv2 could increase performance.
THIS tensor([0.1126], grad_fn=<SelectBackward0>)


In [1]:
from PredictionModel.heatmaps import genheatmappatient

patient_path = 'patient-data/p001'
genheatmappatient(patient_path)

Setting XRayResizer engine to cv2 could increase performance.
Setting XRayResizer engine to cv2 could increase performance.
